In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Cell 1: Configuration
# EITHER: point to Task-1 engineered file (recommended: faster, consistent)
USE_CLEAN_FIRST = True
CLEAN_PATH = "loan_clean_subset.csv"   # upload this if you have it

# OR: fall back to raw file (we'll do a minimal preprocess if clean is absent)
RAW_PATH = "/kaggle/input/shodhh/accepted_2007_to_2018Q4.csv"  # upload if needed
NROWS_FROM_RAW = 200_000    # None for full data (Colab can handle; start smaller if RAM is tight)

RANDOM_STATE = 42

In [3]:
# Cell 2: Install Libraries and Imports
!pip install -U d3rlpy==2.4.0
!pip install -U gymnasium[classic-control]==0.29.1

import os, gc, textwrap, numpy as np, pandas as pd, matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix

import d3rlpy
from d3rlpy.dataset import MDPDataset
from d3rlpy.algos import CQL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 10.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s e

In [6]:
# Cell 3: Helper Functions
DEFAULT_LIKE = {
    "Charged Off","Default",
    "Late (31-120 days)","Late (16-30 days)",
    "Does not meet the credit policy. Status:Charged Off"
}
PAID_LIKE = {"Fully Paid","Does not meet the credit policy. Status:Fully Paid"}

def map_target(status: str):
    if pd.isna(status): return np.nan
    s = str(status).strip()
    if s in PAID_LIKE: return 0
    if s in DEFAULT_LIKE: return 1
    return np.nan

def parse_pct(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series.astype(str).str.rstrip("%"), errors="coerce")

def parse_emp_length(series: pd.Series) -> pd.Series:
    def _map(v):
        if pd.isna(v): return np.nan
        s = str(v).strip().lower()
        if s in ("n/a","na","none"): return np.nan
        if s.startswith("<"): return 0.5
        if "10+" in s: return 10.0
        for tok in s.split():
            try: return float(tok)
            except: pass
        return np.nan
    return series.apply(_map)

FEATURES_RAW = [
    "loan_amnt","funded_amnt","term","installment","int_rate",
    "annual_inc","dti","emp_length","home_ownership","verification_status",
    "purpose","addr_state","revol_bal","revol_util",
    "open_acc","total_acc","delinq_2yrs","inq_last_6mths","pub_rec",
    "issue_d","earliest_cr_line","loan_status"
]

def add_date_features(frame: pd.DataFrame) -> pd.DataFrame:
    X = frame.copy()
    if "issue_d" in X:
        X["issue_year"] = X["issue_d"].dt.year
        X["issue_month"] = X["issue_d"].dt.month
        X["issue_ym"] = X["issue_year"]*12 + X["issue_month"]
    if "earliest_cr_line" in X:
        if "issue_d" in X:
            X["credit_hist_months"] = (
                (X["issue_d"].dt.year - X["earliest_cr_line"].dt.year)*12 +
                (X["issue_d"].dt.month - X["earliest_cr_line"].dt.month)
            )
        else:
            ref = pd.Timestamp("2018-12-01")
            X["credit_hist_months"] = (
                (ref.year - X["earliest_cr_line"].dt.year)*12 +
                (ref.month - X["earliest_cr_line"].dt.month)
            )
    for d in ("issue_d","earliest_cr_line"):
        if d in X: X.drop(columns=[d], inplace=True)
    return X

def robust_time_split(mdf: pd.DataFrame):
    # 80/20 by issue_d if feasible; else stratified random
    if "issue_d" in mdf.columns and mdf["issue_d"].notna().mean() >= 0.7:
        temp = mdf[mdf["issue_d"].notna()].copy()
        cutoff = temp["issue_d"].quantile(0.80)
        train_idx = temp.index[temp["issue_d"] <= cutoff]
        test_idx  = temp.index[temp["issue_d"] >  cutoff]
        train_df = mdf.loc[train_idx].copy()
        test_df  = mdf.loc[test_idx].copy()
        # Attach NaT rows to train to avoid empties (ok for this task)
        nat_rows = mdf.index[mdf["issue_d"].isna()]
        train_df = pd.concat([train_df, mdf.loc[nat_rows]], axis=0)
        if len(train_df)>0 and len(test_df)>0 and train_df["default"].nunique()==2:
            return train_df, test_df
    # Fallback
    tr, te = train_test_split(mdf, test_size=0.2, random_state=42, stratify=mdf["default"])
    return tr.copy(), te.copy()

# Robust trainer that works across d3rlpy versions
def robust_fit(algo, dataset, logdir="./logs_cql", verbose=True):
    attempts = []

    # Most common modern signatures
    attempts.append(lambda: algo.fit(dataset, n_epochs=20, logdir=logdir, verbose=verbose))
    attempts.append(lambda: algo.fit(dataset, epochs=20, logdir=logdir, verbose=verbose))

    # Positional-only epochs (older versions)
    attempts.append(lambda: algo.fit(dataset, 20, logdir=logdir, verbose=verbose))
    attempts.append(lambda: algo.fit(dataset, 20))

    # Step-based training (some versions prefer total gradient steps)
    attempts.append(lambda: algo.fit(dataset, n_steps=100_000, logdir=logdir, verbose=verbose))
    attempts.append(lambda: algo.fit(dataset, total_steps=100_000, logdir=logdir, verbose=verbose))

    # Last resort: minimal call
    attempts.append(lambda: algo.fit(dataset))

    last_err = None
    for i, call in enumerate(attempts, 1):
        try:
            print(f"[robust_fit] Trying signature #{i} ...")
            return call()
        except TypeError as e:
            print(f"[robust_fit] Signature #{i} failed: {e}")
            last_err = e
        except Exception as e:
            print(f"[robust_fit] Signature #{i} failed (other): {e}")
            last_err = e
    raise last_err if last_err else RuntimeError("All fit() signatures failed.")

In [7]:
# Cell 4: Load and Split Data
df_clean = None
if USE_CLEAN_FIRST and os.path.exists(CLEAN_PATH):
    print("[Info] Using clean engineered file:", CLEAN_PATH)
    df_clean = pd.read_csv(CLEAN_PATH)
    assert "default" in df_clean.columns, "Clean file must contain 'default' column."

if df_clean is None:
    print("[Info] CLEAN not found → preprocessing raw:", RAW_PATH)
    hdr = pd.read_csv(RAW_PATH, compression="infer", nrows=0, low_memory=True)
    usecols = [c for c in FEATURES_RAW if c in hdr.columns]
    df = pd.read_csv(RAW_PATH, compression="infer", usecols=usecols, nrows=NROWS_FROM_RAW, low_memory=True)
    print("Raw shape:", df.shape)

    if "int_rate" in df:   df["int_rate"] = parse_pct(df["int_rate"])
    if "revol_util" in df: df["revol_util"] = parse_pct(df["revol_util"])
    if "emp_length" in df: df["emp_length"] = parse_emp_length(df["emp_length"])
    for dcol in ("issue_d","earliest_cr_line"):
        if dcol in df: df[dcol] = pd.to_datetime(df[dcol], format="%b-%Y", errors="coerce")

    df["default"] = df["loan_status"].apply(map_target)
    df = df[~df["default"].isna()].copy()

    # Build modeling frame
    keep_cols = [c for c in df.columns if c != "loan_status"]
    model_df = df[keep_cols].copy()
    train_df, test_df = robust_time_split(model_df)

    X_train_raw = train_df.drop(columns=["default"])
    X_test_raw  = test_df.drop(columns=["default"])
    y_train = train_df["default"].astype(int).values
    y_test  = test_df["default"].astype(int).values

    X_train = add_date_features(X_train_raw)
    X_test  = add_date_features(X_test_raw)

else:
    print("Clean file shape:", df_clean.shape)
    X = df_clean.drop(columns=["default"])
    y = df_clean["default"].astype(int).values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )

print("Train rows:", len(y_train), " Test rows:", len(y_test))
display(X_train.head())

[Info] CLEAN not found → preprocessing raw: accepted_2007_to_2018Q4.csv


FileNotFoundError: [Errno 2] No such file or directory: 'accepted_2007_to_2018Q4.csv'

In [ ]:
# Cell 5: Define Preprocessing Pipeline
num_features = [c for c in X_train.columns if pd.api.types.is_numeric_dtype(X_train[c])]
cat_features = [c for c in X_train.columns if not pd.api.types.is_numeric_dtype(X_train[c])]

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocess = ColumnTransformer([
    ("num", numeric_transformer, num_features),
    ("cat", categorical_transformer, cat_features),
])

X_train_np = preprocess.fit_transform(X_train)
X_test_np  = preprocess.transform(X_test)

print("Final feature dim:", X_train_np.shape[1])

In [ ]:
# Cell 6: Build Offline RL Dataset
# We create one-step transitions for BOTH actions per state:
# - action=1 (Approve) → reward depends on outcome
# - action=0 (Deny)    → reward = 0

def compute_reward(approve: int, default_flag: int, loan_amnt: float, int_rate: float) -> float:
    if approve == 0:
        return 0.0
    # approve == 1
    if default_flag == 0:  # fully paid
        return float(loan_amnt) * float(int_rate) / 100.0  # int_rate is a percent value
    else:  # defaulted
        return -float(loan_amnt)

assert "loan_amnt" in X_train.columns and "int_rate" in X_train.columns, \
    "loan_amnt and int_rate must be present in features for the reward function."

# Build transitions for TRAIN
obs_train = []
act_train = []
rew_train = []
ter_train = []

for i in range(len(y_train)):
    s = X_train_np[i]
    la = X_train.iloc[i]["loan_amnt"]
    ir = X_train.iloc[i]["int_rate"]
    d  = int(y_train[i])

    # action 0 (deny)
    obs_train.append(s); act_train.append(0)
    rew_train.append(compute_reward(0, d, la, ir)); ter_train.append(1.0)

    # action 1 (approve)
    obs_train.append(s); act_train.append(1)
    rew_train.append(compute_reward(1, d, la, ir)); ter_train.append(1.0)

obs_train = np.asarray(obs_train, dtype=np.float32)
act_train = np.asarray(act_train, dtype=np.int64)
rew_train = np.asarray(rew_train, dtype=np.float32)
ter_train = np.asarray(ter_train, dtype=np.float32)

train_dataset = MDPDataset(
    observations=obs_train,
    actions=act_train,
    rewards=rew_train,
    terminals=ter_train,
)

# Build transitions for TEST (for evaluation simulation)
obs_test = []
act_test = []
rew_test = []
ter_test = []

for i in range(len(y_test)):
    s = X_test_np[i]
    la = X_test.iloc[i]["loan_amnt"]
    ir = X_test.iloc[i]["int_rate"]
    d  = int(y_test[i])

    # Same augmentation to allow greedy eval
    obs_test.append(s); act_test.append(0)
    rew_test.append(compute_reward(0, d, la, ir)); ter_test.append(1.0)

    obs_test.append(s); act_test.append(1)
    rew_test.append(compute_reward(1, d, la, ir)); ter_test.append(1.0)

obs_test = np.asarray(obs_test, dtype=np.float32)
act_test = np.asarray(act_test, dtype=np.int64)
rew_test = np.asarray(rew_test, dtype=np.float32)
ter_test = np.asarray(ter_test, dtype=np.float32)

test_dataset = MDPDataset(
    observations=obs_test,
    actions=act_test,
    rewards=rew_test,
    terminals=ter_test,
)

n_train = obs_train.shape[0]
n_test  = obs_test.shape[0]

print(f"Train transitions: {n_train:,}")
print(f"Test transitions : {n_test:,}")
print(f"Unique train states (applications): {n_train // 2:,}")
print(f"Unique test  states (applications): {n_test  // 2:,}")
print("obs_train shape:", obs_train.shape, " actions:", act_train.shape,
      " rewards:", rew_train.shape, " terminals:", ter_train.shape)
print("obs_test  shape:", obs_test.shape,  " actions:", act_test.shape,
      " rewards:", rew_test.shape,  " terminals:", ter_test.shape)

try:
    print("train_dataset.size():", train_dataset.size())
    print("test_dataset.size():",  test_dataset.size())
except Exception:
    pass

In [ ]:
# Cell 7: Define Algorithm
# This cell was added to define the CQL algorithm.
algo = d3rlpy.algos.DiscreteCQL(
    batch_size=32,
    learning_rate=6.25e-05,
    use_gpu=False # Set to True if a GPU is available
)

In [ ]:
# Cell 8: Train Algorithm
# This calls the robust_fit function defined in Cell 3
robust_fit(algo, train_dataset, logdir="./logs_cql", verbose=True)

In [ ]:
# Cell 9: Evaluate Policy Offline on Test Set
# Greedy action per state: a_hat = argmax_a Q(s,a)
# Then compute the realized reward using the ground-truth label.

def greedy_action(algo: CQL, states: np.ndarray) -> np.ndarray:
    # d3rlpy's predict expects batch of observations
    return algo.predict(states)

# Deduplicate the paired (s, a=0) and (s, a=1) in test_dataset
unique_states = obs_test[::2]   # every two entries share the same state
assert unique_states.shape[0] == len(y_test)

a_hat = greedy_action(algo, unique_states)  # 0 or 1 per state

# Compute rewards under your rule
rewards = []
approved_defaults = 0
approved_paid = 0

for i in range(len(y_test)):
    la = X_test.iloc[i]["loan_amnt"]
    ir = X_test.iloc[i]["int_rate"]
    d  = int(y_test[i])
    ah = int(a_hat[i])

    r = compute_reward(ah, d, la, ir)
    rewards.append(r)

    if ah == 1:
        if d == 1: approved_defaults += 1
        else:      approved_paid += 1

rewards = np.array(rewards, dtype=float)

avg_reward = rewards.mean()
approval_rate = float((a_hat == 1).mean())

print(f"Avg reward per application: {avg_reward:,.2f}")
print(f"Approval rate: {approval_rate*100:.2f}%")
print(f"Approved & Fully Paid count: {approved_paid}")
print(f"Approved & Defaulted count : {approved_defaults}")

# Confusion-style table for approvals on test
deny_paid = ((a_hat == 0) & (y_test == 0)).sum()
deny_def  = ((a_hat == 0) & (y_test == 1)).sum()
app_paid  = ((a_hat == 1) & (y_test == 0)).sum()
app_def   = ((a_hat == 1) & (y_test == 1)).sum()

print("\nDecision vs Outcome (test):")
print(pd.DataFrame(
    [[deny_paid, deny_def],
     [app_paid,  app_def]],
    index=["Deny","Approve"],
    columns=["Paid","Default"]
))

In [ ]:
# Cell 10: Save Artifacts
ART_DIR = "/content/offline_rl_cql"
os.makedirs(ART_DIR, exist_ok=True)

# Save d3rlpy model
algo.save(os.path.join(ART_DIR, "cql_discrete_model.d3"))

# Save the sklearn preprocess pipeline
import joblib
joblib.dump(preprocess, os.path.join(ART_DIR, "preprocess.joblib"))

# Save a quick report
with open(os.path.join(ART_DIR, "report.txt"), "w") as f:
    f.write(textwrap.dedent(f"""
    Offline RL — CQL (Discrete) — One-step Loan Approval
    Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}
    Avg reward per application (test): {avg_reward:,.4f}
    Approval rate (test)              : {approval_rate*100:.2f}%
    Approved & Fully Paid (count)     : {approved_paid}
    Approved & Defaulted (count)      : {approved_defaults}
    """).strip())

print("Saved:")
print(" -", os.path.join(ART_DIR, "cql_discrete_model.d3"))
print(" -", os.path.join(ART_DIR, "preprocess.joblib"))
print(" -", os.path.join(ART_DIR, "report.txt"))